# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [65]:
from collections import defaultdict, Counter
import re
from nltk.tokenize import word_tokenize
from nltk.util import ngrams as nltk_ngrams

# Function to load N-gram data from a file
def load_ngrams(filename):
    ngrams = defaultdict(Counter)
    with open(filename, 'r', encoding='ISO-8859-1') as file:
        for line in file:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                ngram, count = parts
                words = ngram.split(' ')
                if len(words) == 2:
                    prefix, word = words
                    ngrams[prefix][word] += int(count)
    return ngrams

# Function to preprocess text by removing headers and footers specific to Project Gutenberg files
def preprocess_text(filename):
    lines = []
    start_reading = False
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith('*** START OF THIS PROJECT GUTENBERG EBOOK'):
                start_reading = True
                continue
            elif line.startswith('*** END OF THIS PROJECT GUTENBERG EBOOK'):
                break
            if start_reading:
                lines.append(line.strip())
    return ' '.join(lines)

# Function to build a bigram model from text
def build_bigram_model(text):
    model = defaultdict(Counter)
    tokens = word_tokenize(text.lower())  # Convert to lowercase and tokenize
    for w1, w2 in nltk_ngrams(tokens, 2, pad_right=True, pad_left=True, left_pad_symbol=None, right_pad_symbol=None):
        model[w1][w2] += 1
    return model

# Function to generate all possible edits that are one edit away from the input word
def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

# Function to filter and return only the known words from a set of candidate words
def known(words, ngrams):
    return set(w for w in words if w in ngrams)

# Function to correct a given word based on its context (previous word) and the ngram model
def correct_word(word, previous_word, ngrams):
    candidates = known([word], ngrams) or known(edits1(word), ngrams) or [word]
    corrected_word = max(candidates, key=lambda candidate: word_probability(candidate, previous_word, ngrams))
    return corrected_word

# Function to calculate the probability of a word given the previous word using the bigram model
def word_probability(word, previous_word, ngrams):
    if previous_word in ngrams and word in ngrams[previous_word]:
        return ngrams[previous_word][word] / sum(ngrams[previous_word].values())
    else:
        return 1 / sum(ngrams[previous_word].values()) if previous_word in ngrams else 0

# Example usage
bigrams = load_ngrams('bigrams.txt')
fivegrams = load_ngrams('fivegrams.txt')  # Example loading, though not used in the given code

# Process and load the 'big.txt' for additional training data
text = preprocess_text('big.txt')
bigram_model_from_text = build_bigram_model(text)

# Combine the models (optional, depending on your use case)
# This simplistic approach adds counts; for more nuanced merging, consider adjusting probabilities
for prev_word, following_words in bigram_model_from_text.items():
    for word, count in following_words.items():
        bigrams[prev_word][word] += count

# Correct an example word
previous_word = 'a'
incorrect_word = 'bae'
corrected_word = correct_word(incorrect_word, previous_word, bigrams)
print(f'Corrected word: {corrected_word}')



Corrected word: bade


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

Use of N-gram Models: The core of the spelling correction system is based on bigram models. This choice allows the system to consider the context of the word being corrected by looking at its immediate predecessor. Bigram models are simple yet effective for capturing local context, which is often sufficient for spelling correction.

DefaultDict and Counter for Model Storage: The use of defaultdict(Counter) for storing n-gram frequencies simplifies the management of counts and probabilities. This data structure choice allows for efficient lookups and updates, which are crucial for performance, especially when working with large text corpora.

Edit Distance Operations in edits1 Function: The implementation of the edits1 function, which generates all possible edits that are one edit away from the input word, uses four types of operations: deletions, transpositions, replacements, and insertions. These operations cover a wide range of common spelling mistakes, making the corrector versatile in handling different types of errors.

Word Probability Calculation: The word_probability function computes the likelihood of a word given its previous word using the frequencies from the bigram model. This probabilistic approach helps in selecting the most likely correction among several candidates by leveraging statistical information from the text data.

Correction Based on Context and Edit Distance: The correct_word function combines contextual information and edit distance operations to find the best correction. It first tries to find the word in the model (zero edits), then looks for one-edit corrections known to the model, and defaults to the original word if no known corrections are found. This prioritization ensures that more probable corrections are favored, and the original word is preserved when no confident correction can be made.

Incorporating Additional Training Data: The code snippet shows an approach to merging bigram models from different sources (e.g., pre-loaded bigrams and those built from a new text corpus). This strategy enhances the corrector's robustness by broadening its vocabulary and contextual understanding, thereby improving its accuracy.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [68]:


# Context-sensitive spelling corrector wrapper
def context_sensitive_wrapper(word, previous_word):
    return correct_word(word, previous_word, bigrams)  

# Evaluation function
def evaluate_spelling_corrector(corrector, test_set):
    correct_count = 0
    for previous_word, incorrect, correct in test_set:

        predicted = corrector(incorrect, previous_word)
        print(predicted )
        if predicted == correct:
            correct_count += 1
    accuracy = correct_count / len(test_set)
    return accuracy

# Test set
test_set_with_context = [
    ('a', 'speling', 'spelling'),  # deletion
    ('the', 'korrectud', 'corrected'),  # substitution
    ('my', 'bycycle', 'bicycle'),  # insertion
    ('very', 'inconvient', 'inconvenient'),  # transposition
    ('they', 'arrainged', 'arranged'),  # mixed
    ('an', 'exampel', 'example'),  # substitution
    ('to', 'writting', 'writing'),  # double letter
    ('a', 'beutiful', 'beautiful'),  # substitution
    ('his', 'responce', 'response'),  # common mistake
    ('we', 'recieve', 'receive')  # ie/ei confusion
]

# Placeholder for context-sensitive accuracy since the actual function is not implemented
accuracy_context_sensitive = evaluate_spelling_corrector(context_sensitive_wrapper, test_set_with_context)

print(f"Accuracy of Context-Sensitive Corrector: {accuracy_context_sensitive:.2f}")

speling
korrectud
bycycle
inconvient
arranged
example
writing
beautiful
response
relieve
Accuracy of Context-Sensitive Corrector: 0.50
